In [9]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Load the CSV file containing image paths and labels
data = pd.read_csv('setdata.csv')

# Path to the images directory
image_dir = 'images/'

# Prepare lists for images and labels
images = []
labels = []

# Load images and labels
for index, row in data.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    image = load_img(img_path, target_size=(32, 32))  # Resize images if necessary
    image = img_to_array(image) / 255.0  # Normalize pixel values
    images.append(image)
    labels.append(row['label'])

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode labels as integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# One-hot encode the labels
labels = to_categorical(labels)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(images)

# Building the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer matches the number of classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on your custom dataset
history = model.fit(datagen.flow(images, labels, batch_size=64),
                    epochs=30,
                    validation_split=0.2,  # Use 20% of the data for validation
                    verbose=2)

# Evaluate the model
test_loss, test_acc = model.evaluate(images, labels, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

# Save the model
model.save('custom_cnn_model.h5')


ModuleNotFoundError: No module named 'tensorflow'